# Answering Business Questions for a Digital Media Store Using SQL

## Introduction
In this project we'll be answering business questions using the "Chinook" data model which represents a digital media store, and contains tables for artists, albums, media tracks, invoices and customers. Media related data was created using real data from an iTunes Library. Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.


## Getting Started
We'll use the following code to connect our Jupyter Notebook to our database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Let's start by getting familiar with our data.

In [2]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The table `employee` contains personal information on employees like their name, address, and position. Employees have employee ID's that are used to link them to the customers they service in the table `customer`. The `customer` table also contains personal information about customers like their names, addresses, and a customer ID that links to the table `invoice`. `Invoice` details billing information for customers, including their invoice total and invoice ID. A single invoice can contain multiple tracks purchased by a customer, the details of which are in the `invoice_line` table which contains invoice_line IDs. The `invoice_line` table also gives us a track ID that can be used to look up the details of the tracks purchased, like the artist ID, album ID, & length of song in the `track` table. Tracks can be composed into a playlist. The `playlist_track` table contains playlist IDs and the track ID's of the songs in each playlist. The table `playlist` contains playlist names and their respective playlist ID's. The `artist` and `album` tables contain information about the artist and albums for tracks. Lastly, tracks may have a genre and media type, which can be found in the `genre` and `media_type` tables.

In [3]:
%%html
<style>
table {float:left}
</style>

## Scenario 1
The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce. The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA. For this scenario we'll need to write a query to find out which genres sell the most tracks in the USA.

|Artist Name|Genre|
|---|---|
|Regal| Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The genre of a track purchased can be found in the table `genre`. We can find information about the number of track's purchased in the `invoice_line` table which tells us the `track ID` and `quantity` purchased. Lastly we'll want to make sure that we're only looking at the quantities of tracks purchased by customers in the USA, so we'll use the `invoice` table to confirm the billing address related to the tracks purchased. For this question we will assume that the billing address is the same as the customer's country of residence.

In [4]:
%%sql

WITH usa_customers AS
    (SELECT invoice_id
       FROM invoice
      WHERE billing_country = "USA"),
    
    usa_tracks AS
    (SELECT track_id,
            quantity
       FROM invoice_line AS il
      INNER JOIN usa_customers AS uc ON uc.invoice_id = il.invoice_id),

    tracks_genre AS
    (SELECT t.track_id AS track_id,
            g.name AS genre
       FROM track AS t
      INNER JOIN genre AS g ON g.genre_id = t.genre_id)

SELECT tg.genre,
       SUM(ut.quantity) AS tracks_sold,
       CAST(SUM(ut.quantity) AS FLOAT)/(SELECT CAST(SUM(quantity) AS FLOAT)
                                          FROM usa_tracks) AS percentage_tracks_sold,
       CASE
       WHEN genre LIKE "%Hop%" THEN "Regal"
       WHEN genre LIKE "%Punk%" THEN "Red Tone"
       WHEN genre LIKE "%Pop%" THEN "Meteor and the Girls"
       WHEN genre LIKE "%Blues%" THEN "Slim Jim Bites"
       ELSE ""
       END AS new_artists
  FROM usa_tracks AS ut
 INNER JOIN tracks_genre AS tg on tg.track_id = ut.track_id
 GROUP BY 1
 ORDER BY 2 DESC;

Done.


genre,tracks_sold,percentage_tracks_sold,new_artists
Rock,561,0.5337773549000951,
Alternative & Punk,130,0.12369172216936251,Red Tone
Metal,124,0.11798287345385347,
R&B/Soul,53,0.05042816365366318,
Blues,36,0.03425309229305423,Slim Jim Bites
Alternative,35,0.03330161750713606,
Latin,22,0.02093244529019981,
Pop,22,0.02093244529019981,Meteor and the Girls
Hip Hop/Rap,20,0.019029495718363463,Regal
Jazz,14,0.013320647002854425,


Based on our output table, Rock tracks made up 53% of tracks sold by Chinook in the USA. Unfortunately, none of our prospective artists fall into this genre. The next most popular genre was Alternative & Punk with 12% of tracks sold. The artist Red Tone falls into this genre. Blues, Pop, and Hip-Hop each respectively make up 2-3% of the total tracks sold. If we can only pick 3 artists, then Red Tone, Slim Jim Bites, and Meteor and the Girls make good selections since they have the highest relative tracks_sold. It's worth noting that there is only a difference of two tracks sold between Regal's genre "Hip Hop/Rap" and Meteor and the Girls' genre of "Pop". With more data we might look into how many social media followers the artists have to add another lens to our analysis. For this exercise, we'll go with a recommendation that the Red Tone, Slim Jim Bites, and Meteor and the Girls albums be added to the Chinook store. We might also recommend that Chinook try to source more Rock and Metal artists to be added to the store in the future.

## Scenario 2

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others. To answer this question, we'll write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. We may also need to consider additional attributes for employees as we get into the analysis. For this exercise we'll use the `employee`, `customer` and `invoice` tables.


In [5]:
%%sql

SELECT e.first_name ||" "|| e.last_name AS employee_name,
       ROUND(SUM(i.total),2) AS total_sales_dollars
 FROM customer AS c
INNER JOIN invoice AS i on i.customer_id = c.customer_id
INNER JOIN employee AS e on e.employee_id = c.support_rep_id
GROUP BY 1;

Done.


employee_name,total_sales_dollars
Jane Peacock,1731.51
Margaret Park,1584.0
Steve Johnson,1393.92


So far we have learned there are three sales support agents. The highest and lowest total sales amount differ only by $338, so it doesn't look like the employee performance differs by much based on sales amount. One reason why Jane Peacock might have sold more than Steve Johnson is that she was hired earlier. Let's test this theory by including the employee `hire_date` in our output.

In [6]:
%%sql

SELECT e.first_name ||" "|| e.last_name AS employee_name,
       e.hire_date AS hire_date,
       ROUND(SUM(i.total),2) AS total_sales_dollars
 FROM customer AS c
INNER JOIN invoice AS i on i.customer_id = c.customer_id
INNER JOIN employee AS e on e.employee_id = c.support_rep_id
GROUP BY 1;

Done.


employee_name,hire_date,total_sales_dollars
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Our hypothesis is true. Steve was hired latest in the year and thus has sold the least. In response to the question, our answer is that no sales support agent is performing better or worse than the others.

## Scenario 3

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Since there are a number of countries with only 1 customer, we will group these customers as "Other" in our analysis.

In [7]:
%%sql

WITH sales_country AS
      (SELECT c.customer_id AS customer_id,
              c.country AS country,
              ROUND(SUM(i.total),2) AS total
        FROM customer AS c
       INNER JOIN invoice AS i on i.customer_id = c.customer_id
       GROUP BY 1
       ORDER BY 1),
    
     avg_order AS
     (SELECT c.country, 
             ROUND(AVG(i.total),2) AS avg_order_value
        FROM invoice AS i
       INNER JOIN customer AS c on c.customer_id = i.customer_id
       GROUP BY 1),
    
    country_all AS 
    (SELECT ao.country,
            COUNT(sc.customer_id) AS num_customers,
            sc.total,
            ROUND(sc.total/COUNT(sc.customer_id),2) AS sales_per_customer,
            avg_order_value
       FROM sales_country AS sc
      INNER JOIN avg_order AS ao ON sc.country = ao.country
      GROUP BY 1),
  
    other AS
    (SELECT CASE
            WHEN num_customers = 1 THEN "Other"
            END AS country,
            SUM(num_customers) AS num_customers,
            SUM(total) AS total,
            ROUND(AVG(sales_per_customer),2) AS sales_per_customer,
            ROUND(AVG(avg_order_value),2) AS avg_order_value
       FROM country_all
      WHERE num_customers = 1),
     
    country_with_other AS   
    (SELECT *
       FROM country_all
      WHERE num_customers > 1

     UNION

     SELECT *
       FROM other
      ORDER BY total DESC),
    
    country_sort AS
    (SELECT *,
            CASE
            WHEN country = "Other" THEN 1
            ELSE 0
            END AS sort
       FROM country_with_other)

SELECT country, 
       num_customers,
       total,
       sales_per_customer,
       avg_order_value
 FROM country_sort
ORDER BY sort;

Done.


country,num_customers,total,sales_per_customer,avg_order_value
Czech Republic,2,128.7,64.35,9.11
Brazil,5,106.92,21.38,7.01
Portugal,2,82.17,41.09,6.38
United Kingdom,3,79.2,26.4,8.77
Canada,8,75.24,9.4,7.05
France,5,73.26,14.65,7.78
Germany,4,73.26,18.32,8.16
USA,13,72.27,5.56,7.94
India,2,71.28,35.64,8.72
Other,15,1094.94,73.0,7.44


## Scenario 4

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

**Additional background information:**

- The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). 
- When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

**Known edge cases:**

 - Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.

Since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks.

 - Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.
 
The company has previously done analysis to confirm this case does not happen often, so we can ignore this case also.

**Approach**

To address this question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [14]:
%%sql

WITH invoice_track AS

    (SELECT invoice_id, track_id
       FROM invoice_line
   GROUP BY invoice_id)

SELECT album_purchase,
       COUNT(invoice_id) AS num_invoices,
       CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(invoice_id)
                                           FROM invoice) percent

 FROM (SELECT it.*,
         CASE
         WHEN
                (
                 SELECT track_id 
                   FROM track AS t
                  WHERE album_id = (SELECT t2.album_id
                                    FROM track AS t2
                                   WHERE track_id = it.track_id)

             EXCEPT

                SELECT track_id
                  FROM invoice_line AS il
                 WHERE invoice_id = it.invoice_id
                ) IS NULL

        AND 

                (
                SELECT track_id
                  FROM invoice_line AS il
                 WHERE invoice_id = it.invoice_id

                EXCEPT 

                SELECT track_id 
                  FROM track AS t
                WHERE album_id = (SELECT t2.album_id
                                    FROM track AS t2
                                   WHERE track_id = it.track_id)
                ) IS NULL

        THEN "yes"
        ELSE "no"
        END AS album_purchase
        FROM invoice_track AS it)

GROUP BY album_purchase;

Done.


album_purchase,num_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Since 81% of invoices are not album purchases, it is recommedned that the Chinook store not continue to buy full albums from record companies, and instead just focus on purchasing the most popular tracks from albums.